Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [18]:
project_dir = '..\\..\\..\\..\\'

In [80]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from average_precision import apk
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(dict_of_preds, dict_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    # loop over the items of the customers in both sets
    for c_id, preds in dict_of_preds.items():
        gt = dict_of_gts.get(c_id, [])
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [20]:
import pandas as pd

In [21]:
%%time

transactions = pd.read_parquet(f'{project_dir}Input\\Dataset\\transactions_train.parquet')
customers = pd.read_parquet(f'{project_dir}Input\\Dataset\\customers.parquet')
articles = pd.read_parquet(f'{project_dir}Input\\Dataset\\articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: total: 1.95 s
Wall time: 1.06 s


In [22]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [23]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: total: 12.1 s
Wall time: 18 s


In [24]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [25]:
c2weeks

customer_id
28847241659200          [95, 96, 101, 102]
41318098387474                        [98]
116809474287335                 [101, 103]
200292573348128          [95, 96, 99, 102]
248294615847351                       [96]
                               ...        
18446624797007271432                  [95]
18446630855572834764                 [103]
18446662237889060501                 [100]
18446705133201055310                 [102]
18446737527580148316                 [104]
Name: week, Length: 437365, dtype: object

In [26]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: total: 234 ms
Wall time: 641 ms


In [27]:
c2weeks2shifted_weeks[28847241659200]
# todo ??

{95: 96, 96: 101, 101: 102, 102: 105}

In [28]:
candidates_last_purchase = transactions.copy()

In [29]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: total: 12.9 s
Wall time: 20.6 s


In [30]:
candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
29319533,2020-07-22,272412481300040,885077001,0.008458,1,103
29410772,2020-07-24,272412481300040,850176003,0.029034,2,103
29410773,2020-07-24,272412481300040,875803001,0.064559,2,103
29410774,2020-07-24,272412481300040,892970003,0.020966,2,103
29410775,2020-07-24,272412481300040,854619003,0.020966,2,103


In [31]:
# transactions[transactions['customer_id']==272412481300040]
customers[customers['customer_id']==272412481300040]

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
137308,272412481300040,1,1,0,1,48,333369


### Bestsellers candidates

In [32]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [33]:
mean_price

week  article_id
95    108775015     0.004729
      108775044     0.008458
      110065001     0.006085
      110065002     0.006085
      111565001     0.004288
                      ...   
104   952267001     0.013732
      952938001     0.048651
      953450001     0.016932
      953763001     0.021885
      956217002     0.059068
Name: price, Length: 196880, dtype: float32

In [34]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [35]:
""" query to get most recent items:
    select article_id, min(week) as week, rank() over (partition by article_id order by week desc) as recency_rank
    from transactions 
    group by article_id
    order by week desc    
"""
# query implemented in code
recency = transactions \
    .groupby('article_id')['week'].min()\
    .sort_values(ascending=False).rename('recency_rank').astype('int8')\
    .rank(method='dense', ascending=False) \
    # .groupby('recency_week').head(12).rename('recency_rank').astype('int8')
    

recency



article_id
746586002     1.0
703366001     1.0
889974002     1.0
889901002     1.0
889844001     1.0
             ... 
792517001    10.0
792515001    10.0
792507001    10.0
792490002    10.0
805370003    10.0
Name: recency_rank, Length: 38331, dtype: float64

In [36]:
# add a column with mean_price, min_price, max_price, std_price for each article_id
mean_price_by_article =  transactions \
    .groupby(['article_id'])['price'].mean().rename('mean_price')
min_price_by_article =  transactions \
    .groupby(['article_id'])['price'].min().rename('min_price')
max_price_by_article =  transactions \
    .groupby(['article_id'])['price'].max().rename('max_price')
std_price_by_article =  transactions \
    .groupby(['article_id'])['price'].std().rename('std_price')
# merge
price_statistics = pd.merge(mean_price_by_article, min_price_by_article, on=['article_id']).merge(max_price_by_article, on=['article_id']).merge(std_price_by_article, on=['article_id'])
price_statistics



,mean_price,min_price,max_price,std_price
article_id,,,,
108775015,0.003842,0.002068,0.005068,0.001573
108775044,0.008076,0.003373,0.008458,0.001120
110065001,0.006424,0.006085,0.006763,0.000479
110065002,0.005860,0.005186,0.006085,0.000449
110065011,0.009305,0.006763,0.011847,0.003595
...,...,...,...,...
952267001,0.014982,0.010153,0.016932,0.002385
952938001,0.048006,0.040661,0.050831,0.004300
953450001,0.016836,0.015305,0.016932,0.000395


In [37]:
sales

week  article_id
95    760084003      1
      866731001      2
      600886001      3
      706016001      4
      372860002      5
                    ..
104   915529003      8
      915529005      9
      448509014     10
      762846027     11
      714790020     12
Name: bestseller_rank, Length: 120, dtype: int8

In [38]:
sales.loc[95]

article_id
760084003     1
866731001     2
600886001     3
706016001     4
372860002     5
610776002     6
877278002     7
547780003     8
817354001     9
827968001    10
866731003    11
866383006    12
Name: bestseller_rank, dtype: int8

In [39]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [40]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [41]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [42]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
29030503,2020-07-15,272412481300040,1,95
29064059,2020-07-15,1456826891333599,1,95
29067103,2020-07-15,2133687643102426,2,95
29027487,2020-07-15,6010692573790711,1,95
29046403,2020-07-15,6171059100114610,2,95
...,...,...,...,...
31760188,2020-09-22,18435221511488011015,1,104
31782234,2020-09-22,18436859303155335645,1,104
31787251,2020-09-22,18437941771381362708,2,104
31776022,2020-09-22,18438270306572912089,1,104


In [43]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [44]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [45]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [46]:
test_set_transactions

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105
...,...,...,...,...
437360,2020-09-22,18410229429441241008,2,105
437361,2020-09-22,18417769707947924979,2,105
437362,2020-09-22,18418054986721795659,2,105
437363,2020-09-22,18421175435799911749,2,105


In [47]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [48]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [49]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193
...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,1,105,915529003,0.033439
5248376,2020-09-22,18438270306572912089,1,105,915529005,0.033417
5248377,2020-09-22,18438270306572912089,1,105,448509014,0.041630
5248378,2020-09-22,18438270306572912089,1,105,762846027,0.025005


# Combining transactions and candidates / negative examples

In [50]:
transactions['purchased'] = 1

In [51]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [52]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0
...,...,...,...,...,...,...,...
5248375,2020-09-22,18438270306572912089,915529003,0.033439,1,105,0.0
5248376,2020-09-22,18438270306572912089,915529005,0.033417,1,105,0.0
5248377,2020-09-22,18438270306572912089,448509014,0.041630,1,105,0.0
5248378,2020-09-22,18438270306572912089,762846027,0.025005,1,105,0.0


In [53]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [54]:
data.purchased.mean()

0.13607582749165664

### Add bestseller information

In [55]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
).merge(
    recency,
    on='article_id',
    how='left'
).merge(
    price_statistics,
    on='article_id',
    how='left'
)


In [56]:

data.head(10)

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,recency_rank,mean_price,min_price,max_price,std_price
0,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0,NaN,10.0,0.005620,0.001678,0.008458,0.002007
1,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0,NaN,10.0,0.016344,0.010153,0.016932,0.001248
2,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0,NaN,10.0,0.033032,0.027102,0.033881,0.001641
3,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0,NaN,10.0,0.024747,0.018627,0.025407,0.001481
4,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0,NaN,10.0,0.024468,0.012695,0.025407,0.001917
5,2020-07-15,1456826891333599,888294001,0.013542,1,95,1.0,NaN,10.0,0.022126,0.006763,0.025407,0.005261
6,2020-07-15,1456826891333599,895002002,0.015237,1,95,1.0,NaN,10.0,0.012476,0.003373,0.015237,0.002595
7,2020-07-15,2133687643102426,843642001,0.042356,2,95,1.0,NaN,10.0,0.041549,0.029153,0.042356,0.002169
8,2020-07-15,6010692573790711,857812010,0.039661,1,95,1.0,NaN,10.0,0.034969,0.013542,0.042356,0.007702
9,2020-07-15,6010692573790711,372860001,0.012678,1,95,1.0,NaN,10.0,0.013197,0.003373,0.013542,0.000962


In [57]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [58]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [59]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [60]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [61]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [62]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active', 'has_color',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank', 'recency_rank', 'std_price']

# columns_to_use += ['mean_price', 'min_price', 'max_price']
# columns_to_use += ['sales_channel_id']

In [63]:
%%time
train_X = train[columns_to_use]
print(train_X.columns)
train_y = train['purchased']

test_X = test[columns_to_use]

Index(['article_id', 'product_type_no', 'graphical_appearance_no',
       'colour_group_code', 'perceived_colour_value_id',
       'perceived_colour_master_id', 'department_no', 'index_code',
       'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
       'has_color', 'club_member_status', 'fashion_news_frequency', 'age',
       'postal_code', 'bestseller_rank', 'recency_rank', 'std_price'],
      dtype='object')
CPU times: total: 344 ms
Wall time: 525 ms


# Model training

In [64]:
from lightgbm.sklearn import LGBMRanker

In [65]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [70]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
    eval_set=[(test_X, test['purchased'])],
    eval_group=[test.groupby(['week', 'customer_id'])['article_id'].count().values],
)


[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.848850
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.178935
[LightGBM] [Debug] init for col-wise cost 0.177210 seconds, init for row-wise cost 0.389924 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.344955 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1346
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 21
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
CPU times: total: 24 s
Wall time: 9.25 s


In [72]:
# Get the predictions for the test week based upon the last week of the training set
last_week_validation = train_X[train.week == train.week.max()]

val_predictions = ranker.predict(last_week_validation)

# Execute mean average precision at k (MAP@k) evaluation between the predictions and the ground truth
c_id2predicted_article_ids = last_week_validation \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()
preds = []
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()
sub = pd.read_csv(f'{project_dir}Input\\Dataset\\sample_submission.csv')
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

test_week_purchases_by_cust = test.groupby('customer_id')['article_id'].apply(list).to_dict()
calculate_apk(dict_of_gts=test_week_purchases_by_cust, dict_of_preds=c_id2predicted_article_ids)

KeyError: 'customer_id'

In [73]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9983298445891587
recency_rank 0.000893660433445673
age 0.0002761468681148894
article_id 0.00011181282516824868
section_no 7.071485964406431e-05
postal_code 6.204015426479855e-05
club_member_status 5.7471018770658855e-05
garment_group_no 4.762275783972834e-05
std_price 4.407661973929278e-05
colour_group_code 3.3416700263611705e-05
department_no 2.9359838951292078e-05
Active 1.7295701569078493e-05
product_type_no 1.5111023305836356e-05
fashion_news_frequency 1.1426609764090385e-05
FN 0.0
has_color 0.0
index_group_no 0.0
index_code 0.0
perceived_colour_master_id 0.0
perceived_colour_value_id 0.0
graphical_appearance_no 0.0


# Calculate predictions

In [81]:
%time
# get the items purchased in the last week before the test week
last_week_purchases = data[data.week == data.week.max() - 1]
# predict the items purchased in the test week 
last_week_purchases['preds'] = ranker.predict(last_week_purchases[columns_to_use])
# calculate the average precision at k for the test week
c_id2predicted_article_ids_validation = last_week_purchases \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()
test_week_purchases_by_cust_validation = test.groupby('customer_id')['article_id'].apply(list).to_dict()
a = calculate_apk(dict_of_gts=test_week_purchases_by_cust_validation, dict_of_preds=c_id2predicted_article_ids_validation)
print(a)

CPU times: total: 0 ns
Wall time: 0 ns


C:\Users\samro\AppData\Local\Temp\ipykernel_7512\4074737826.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_week_purchases['preds'] = ranker.predict(last_week_purchases[columns_to_use])


0.48067903662208145


In [ ]:

test['preds'] = ranker.predict(test_X)


# c_id2predicted_article_ids = test \
#     .sort_values(['customer_id', 'preds'], ascending=False) \
#     .groupby('customer_id')['article_id'].apply(list).to_dict()
# test_week_purchases_by_cust = test.groupby('customer_id')['article_id'].apply(list).to_dict()

# a = calculate_apk(dict_of_gts=test_week_purchases_by_cust, dict_of_preds=c_id2predicted_article_ids)
# print(a)

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

# Create submission

In [55]:
sub = pd.read_csv(f'{project_dir}Input\\Dataset\\sample_submission.csv')

In [56]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: total: 2.77 s
Wall time: 3.74 s


In [57]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [58]:
sub_name = 'basic_model_submission'
sub.to_csv(f'{project_dir}Output\\{sub_name}.csv', index=False)